In [1]:
# 영상 분류 cnn 모델 생성
# CCPD CCPD F FC D FC
# CIFAR-10

In [2]:
# CNN 모델 생성 ~~ 학습

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [13]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [5]:
# 신경망 모델설계
cnn = Sequential()
cnn.add(Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(Conv2D(32,(3,3),activation='relu'))
cnn.add(MaxPooling2D(2,2))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(MaxPooling2D(2,2))
cnn.add(Dropout(0.5))
cnn.add(Flatten())
cnn.add(Dense(512,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))      
        

In [6]:
cnn.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,optimizer=Adam(0.0001),metrics=['acc'])
earlyStop = tf.keras.callbacks.EarlyStopping(patience=3)
hist = cnn.fit(x_train,y_train,batch_size=128,epochs=100, validation_data = [x_test,y_test],callbacks=earlyStop)
cnn.evaluate(x_test,y_test)
cnn.save("my_cnn_for_deply.h5")

In [10]:
# 학습된 이미지를 불러와서 폴더에 담겨있는 영상을 인식

In [14]:
cnn = tf.keras.models.load_model('my_cnn_for_deply.h5')

In [49]:
class_name=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [50]:
from glob import glob

In [67]:
pwd()

'E:\\7.ai'

In [68]:
# os.chdir('E:\\7.ai')
files = glob('./test_images/*.jpg')
files

['./test_images\\01.jpg',
 './test_images\\02.jpg',
 './test_images\\03.jpg',
 './test_images\\04.jpg',
 './test_images\\05.jpg',
 './test_images\\06.jpg',
 './test_images\\07.jpg',
 './test_images\\08.jpg',
 './test_images\\09.jpg',
 './test_images\\10.jpg',
 './test_images\\11.jpg',
 './test_images\\12.jpg']

In [69]:
from tensorflow.keras.preprocessing import image

In [70]:
x_train = []
origin_img = []
for file in files:
    origin_img.append(image.load_img(file)) # 원본 이미지
    img = image.load_img(file,target_size=(32,32) )    
    x = image.img_to_array(img)    
    x_train.append(x)
x_train = (np.array(x_train)).astype('float32')/255.0

In [71]:
x_train.shape

(12, 32, 32, 3)

In [72]:
# 예측
pred = cnn.predict(x_train)

1/1 [==============================] - 0s 22ms/step


In [73]:
pred[0]

array([9.9854112e-01, 5.3825857e-05, 4.6923733e-04, 1.1948367e-05,
       3.1353094e-04, 2.3395884e-07, 8.1546232e-06, 7.0352883e-07,
       5.1177613e-04, 8.9523499e-05], dtype=float32)

In [74]:
import os
#현재경로를 test_images로 이동
os.chdir('./test_images')
# test_images 밑에 class_buckets 폴더가 없으면 생성
if not os.path.isdir('class_buckets'):
    os.mkdir('class_buckets')
os.chdir('./class_buckets')    # 생성한 바구니폴더로 이동한다

# cnn모델이 분류하는 10개의 분류명으로 폴더를 만든다
for i in range(len(class_name)):
    if not os.path.isdir(class_name[i]):
        os.mkdir(class_name[i])

In [75]:
pwd()

'E:\\7.ai\\test_images\\class_buckets'

In [76]:
class_name[ np.argmax(pred[0])]

'airplane'

In [77]:
# cnn 이 인식한 해당 이미지의 결과를 폴더에 저장
for i in range(len(x_train)):
    foler_name = class_name[ np.argmax(pred[i])]
    os.chdir(foler_name)
    origin_img[i].save(foler_name+str(i)+'.jpg')
    os.chdir('../')# 현재폴더에서 바로 상위 폴더로 이동...     ./ 현재경로  ../ 한칸위    ../../ 두칸위    

In [78]:
# 기능 확장